In [4]:
%cd /content/drive/MyDrive/multi/0525

/content/drive/MyDrive/multi/0525


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [74]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [75]:
from tensorflow.keras.datasets import cifar100
(X_train, y_train), (X_test, y_test) = cifar100.load_data()

In [76]:
# X_data: scaling
X_train = X_train/255.
X_test = X_test/255.

In [77]:
# y_data: one hot encoding
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

- 모델 정의/설정

In [78]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import RMSprop

In [79]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3), (50000, 100), (10000, 100))

In [89]:
model = Sequential([ 
    Conv2D(32, (3,3), input_shape=X_train.shape[1:], padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.25),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(),
    Dropout(0.25),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(100, activation='softmax')
])
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_26 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 16, 16, 32)       0         
 g2D)                                                            
                                                                 
 dropout_24 (Dropout)        (None, 16, 16, 32)        0         
                                                                 
 conv2d_27 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 conv2d_28 (Conv2D)          (None, 14, 14, 128)       73856     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 7, 7, 128)        0         
 g2D)                                                 

In [90]:
model.compile(
    optimizer=RMSprop(learning_rate=0.0001, decay=1e-6),
    loss='categorical_crossentropy', metrics=['accuracy']
)

In [91]:
model_path = './best-cifar100.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=10)

In [92]:
hist = model.fit(
    X_train, y_train, validation_split=0.2,
    epochs=100, batch_size=32, callbacks=[mc, es]
)

Epoch 1/100
1248/1250 [============================>.] - ETA: 0s - loss: 4.3310 - accuracy: 0.0457
Epoch 1: val_loss improved from inf to 3.97176, saving model to ./best-cifar100.h5
1250/1250 [==============================] - 25s 20ms/step - loss: 4.3307 - accuracy: 0.0458 - val_loss: 3.9718 - val_accuracy: 0.1097
Epoch 2/100
1248/1250 [============================>.] - ETA: 0s - loss: 3.8522 - accuracy: 0.1149
Epoch 2: val_loss improved from 3.97176 to 3.60367, saving model to ./best-cifar100.h5
1250/1250 [==============================] - 26s 21ms/step - loss: 3.8522 - accuracy: 0.1148 - val_loss: 3.6037 - val_accuracy: 0.1726
Epoch 3/100
1250/1250 [==============================] - ETA: 0s - loss: 3.5722 - accuracy: 0.1618
Epoch 3: val_loss improved from 3.60367 to 3.37727, saving model to ./best-cifar100.h5
1250/1250 [==============================] - 25s 20ms/step - loss: 3.5722 - accuracy: 0.1618 - val_loss: 3.3773 - val_accuracy: 0.2121
Epoch 4/100
1249/1250 [==================

In [93]:
class_names=['apple','aquarium_fish','baby','bear','beaver',
             'bed','bee','beetle','bicycle','bottle','bowl',
             'boy','bridge','bus','butterfly','camel','can',
             'castle','caterpillar','cattle','chair','chimpanzee',
             'clock','cloud','cockroach','couch','crab','crocodile',
             'cup','dinosaur','dolphin','elephant','flatfish','forest',
             'fox','girl','hamster','house','kangaroo','keyboard','lamp',
             'lawn_mower','leopard','lion','lizard','lobster','man','maple_tree',
             'motorcycle','mountain','mouse','mushroom','oak_tree','orange',
             'orchid','otter','palm_tree','pear','pickup_truck','pine_tree',
             'plain','plate','poppy','porcupine','possum','rabbit','raccoon',
             'ray','road','rocket','rose','sea','seal','shark','shrew','skunk',
             'skyscraper','snail','snake','spider','squirrel','streetcar',
             'sunflower','sweet_pepper','table','tank','telephone','television',
             'tiger','tractor','train','trout','tulip','turtle','wardrobe',
             'whale','willow_tree','wolf','woman','worm']

In [94]:
best_model = load_model('./best-cifar100.h5')
best_model.evaluate(X_test, y_test)

313/313 [==============================] - 2s 6ms/step - loss: 2.2437 - accuracy: 0.4360


[2.2437100410461426, 0.4359999895095825]

In [95]:
from PIL import Image

In [106]:
y_pred = best_model.predict(X_test)

In [121]:
len(y_test)

10000

In [127]:
N = 100
ans = 0.

for i in range(N):

    ans = np.where(y_test[i] == 1)[0][0]
    pred = y_pred[i].argmax()

    # print(f'정답 : {class_names[ans]}, 예측 : {class_names[pred]}')
    if class_names[ans] == class_names[pred]:
        ans += 1

print(ans / N)

0.35


In [2]:
!jupyter nbconvert './cifar-100.ipynb' --to html 

[NbConvertApp] WARNING | pattern 'cifar-100.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
    E